In [1]:
import numpy as np
from sklearn.cluster import MeanShift,estimate_bandwidth
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from sklearn.datasets import make_blobs

from sklearn import preprocessing
import pandas as pd

In [2]:
df = pd.read_excel('max_similarity_result.xlsx')
print(df.head(5))

                                             Content  similarity  Useless
0  @AlanDersh I’d believe an abused woman’s word ...    0.202623        0
1  RT @drdavidsamadi: .@CNN, you need to explain ...    0.235064        0
2  RT @KerrieNation: Stop romanticizing violent w...    0.235873        0
3  RT @Slarothestoic: Hello, @barnardos Your ad i...    0.214735        0
4  @realDonaldTrump @FoxNews They must not be rep...    0.213659        0


In [3]:
org_df = pd.DataFrame.copy(df)

In [4]:
print(df.head())

                                             Content  similarity  Useless
0  @AlanDersh I’d believe an abused woman’s word ...    0.202623        0
1  RT @drdavidsamadi: .@CNN, you need to explain ...    0.235064        0
2  RT @KerrieNation: Stop romanticizing violent w...    0.235873        0
3  RT @Slarothestoic: Hello, @barnardos Your ad i...    0.214735        0
4  @realDonaldTrump @FoxNews They must not be rep...    0.213659        0


In [5]:
df.fillna(0, inplace=True)  # 把NaN替换为0

In [6]:
# 把字符串映射为数字，例如{female:1, male:0}

In [7]:
df_map = {}  # 保存映射
cols = df.columns.values
for col in cols:
    if df[col].dtype != np.int64 and df[col].dtype != np.float64:
        temp = {}
        x = 0
        for ele in set(df[col].values.tolist()):
            if ele not in temp:
                temp[ele] = x
                x += 1

        df_map[df[col].name] = temp
        df[col] = list(map(lambda val: temp[val], df[col]))

for key, value in df_map.items():
   print(key,value)
# print(df.head())

Content {'RT @alamedamark: @DrDinD Listening to Trump got this poor kid killed.   https://t.co/62oHzKwBqB': 0, "RT @The_SC_Podcast: If some how you've missed series 1 of The Sports Coaching Podcast you catch catch episodes 1 - 12 here 👉👉https://t.co/R ": 1, 'RT @DollarVigilante: “No society wants you to become wise: it is against the investment of all societies" https://t.co/BCxiIbtQAP': 2, 'RT @Slarothestoic: Hello, @harvardmed Your ad is on the YouTube channel of Rebel News, which calls social distancing measures to fight the ': 3, 'RT @fisher_tx14: This is one of the things we should be talking about. https://t.co/emb2Hsdvzd': 4, 'RT @hannahhalfpint_: If everywhere is a risk why are we opening places that are non essential and making it greater than it needs to be? Ec ': 5, 'RT @CorinneDinDin: If you’re driving in north London today, avoid Brent Cross and the A406. It’s a river... https://t.co/C0jnLAumGp': 6, 'RT @washingtonpost: Companies are hiring their own epidemiologists to hel

In [8]:
# 由于是非监督学习，不使用label

In [9]:
x = np.array(df.drop(['Useless'], 1).astype(float))

In [10]:
# 将每一列特征标准化为标准正太分布，注意，标准化是针对每一列而言的

In [11]:
# Compute Affinity Propagation
x = preprocessing.scale(x)

clf = AffinityPropagation(preference=-50)
clf.fit(x)
cluster_centers_indices = clf.cluster_centers_indices_
labels = clf.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)

org_df['group'] = np.nan
for i in range(len(x)):
    org_df['group'].iloc[i] = labels[i]

contents = {}
for i in range(n_clusters_):
    temp_df = org_df[org_df['group'] == float(i)]
    content_cluster = temp_df[(temp_df['Useless'] == 1)]
    content = 1.0 * len(content_cluster) / len(temp_df)
    contents[i] = content
print(contents)

Estimated number of clusters: 3
{0: 0.0, 1: 0.0, 2: 0.0}


d:\python3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:152: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 0.25 which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning)
d:\python3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
# MeanShift自动把数据分组
# 你可以详细分析一下org_df, 看看这几组人的共同特点是什么
# print(org_df[ org_df['group'] == 2 ])
# print(org_df[ org_df['group'] == 2 ].describe())
org_df.to_excel('1122334455667788.xls')